In [1]:
%load_ext autoreload
%autoreload 1
%matplotlib inline

In [2]:
import os, sys
from os.path import join as pjoin
import json
import numpy as np
import h5py
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt

sys.path.insert(0, pjoin(os.path.pardir, 'webapi'))
sys.path.insert(0, pjoin(os.path.pardir, os.path.pardir, 'mcdose'))
%aimport mcdose
from mcdose.visualize import vis_slice, create_volume_dose_figure
%aimport maskio

ModuleNotFoundError: No module named 'absl'

In [9]:
fmapsfile = '/media/hdd1/g4sim/beamlet_dose/HN011/fmaps_v2.h5'
maskfile = '/media/hdd1/g4sim/beamlet_dose/HN011/masks.h5'
bboxfile = '/media/hdd1/g4sim/beamlet_dose/HN011/cropbox.json'

with open(bboxfile, 'r') as fd:
    bbox = json.load(fd)
    print(bbox)
    
masks = maskio.Mask.load_masks_from_file(maskfile)

{'spacing': [2.5, 2.5, 2.5], 'start': [-240.0, -364.0, -657.0], 'size': [192, 113, 111]}


In [10]:
%aimport planio
from planio import OptimizedPlan

plans = OptimizedPlan.load_plans_from_file('/media/hdd1/g4sim/planning/DoseInfo.mat')
print(plans.keys())

dict_keys(['HN011 1.5T_1e5 Info10 eta1000', 'HN011 1.5T_2000 Info10 eta1000', 'HN011 0.0T_1e5 Info10 eta1000', 'HN011 1.5T_2000_predicted Info10 eta1000'])


In [11]:
# Mask dose and compute statistics
plans['HN011 1.5T_1e5]


SyntaxError: EOL while scanning string literal (<ipython-input-11-2020980c4499>, line 2)

In [56]:
# format statistics
class struct():
    def __init__(self):
        self.name = None
        self.volume = None
        self.mean = None
        self.max = None
        self.min = None
        
def load_stats(f, plan_labels):
    with open(f, 'r') as fd:
        n = 0
        for line in fd:
            n += 1
        nlineper = 8
        nstructs = (n//nlineper//len(plan_labels))

        plans = {k: None for k in plan_labels}
        fd.seek(0)
        for plan in plans.keys():
            structs = {}
            for structnum in range(nstructs):
                st = struct()
                fd.readline()
                st.name = fd.readline().strip('\n').replace('Structure is:  ', '')
                line = fd.readline().strip('\n').replace('Total volume is:  ', '')
                st.volume = (float(line[:line.find(' ')]), line[line.find(' ')+1:])
                fd.readline()
                st.mean = float(fd.readline().strip('\n').replace('Mean dose is:  ', ''))
                st.max = float(fd.readline().strip('\n').replace('Max dose is:  ', ''))
                st.min = float(fd.readline().strip('\n').replace('Min dose is:  ', ''))
                fd.readline()
                structs[st.name] = st
            plans[plan] = structs
        return plans
        
root = '/media/shengnas/SharedProjectData/DeepLearningMC/phase3-planning_study/HN011/results/figures_history_2000_eta_1000_date_02-Mar-2020 18_03_19/rn_figures'
stats = dict(
    true_noisy = load_stats(pjoin(root, 'stats_True_GT-Noisy-Pred.txt'), ['GT', 'Noisy', 'Pred']),
    true_0T    = load_stats(pjoin(root, 'stats_True_GT-0T-Pred.txt'),    ['GT', '0T', 'Pred']),
    plan_noisy = load_stats(pjoin(root, 'stats_Plan_GT-Noisy-Pred.txt'), ['GT', 'Noisy', 'Pred']),
    plan_0T    = load_stats(pjoin(root, 'stats_Plan_GT-0T-Pred.txt'),    ['GT', '0T', 'Pred']),
)
structure_names = set()
for st in stats['true_noisy']['GT'].keys():
    structure_names.add(st)

for dosetype, collection in [('True Dose',     list(stats['true_noisy'].items())+[('0T', stats['true_0T']['0T'])]), 
                             ('Planning Dose', list(stats['plan_noisy'].items())+[('0T', stats['plan_0T']['0T'])])]:
    print(dosetype)
    print('---------')
    for st in structure_names:
        print('"{name}" ({volume} {volunit})'.format(name=st, volume=stat[st].volume[0], volunit=stat[st].volume[1]))
        for label, stat in collection:
            print('  plan: {}'.format(label))
            print('    mean: {}'.format(stat[st].mean))
            print('    max:  {}'.format(stat[st].max))
            print('    min:  {}'.format(stat[st].min))
        print()



True Dose
---------
"L parotid" (26.8125 cubic cm.)
  plan: GT
    mean: 14.8325
    max:  31.9786
    min:  1.3625
  plan: Noisy
    mean: 18.8499
    max:  42.9072
    min:  0.96783
  plan: Pred
    mean: 15.0591
    max:  36.5011
    min:  1.3722
  plan: 0T
    mean: 14.1518
    max:  37.2119
    min:  1.2124

"R cochlea" (0.15625 cubic cm.)
  plan: GT
    mean: 58.0221
    max:  69.0657
    min:  49.6788
  plan: Noisy
    mean: 61.3025
    max:  70.6111
    min:  52.9141
  plan: Pred
    mean: 60.9112
    max:  70.9386
    min:  51.3816
  plan: 0T
    mean: 57.7379
    max:  69.4317
    min:  49.246

"L eye" (14.75 cubic cm.)
  plan: GT
    mean: 7.442
    max:  44.3215
    min:  0.55541
  plan: Noisy
    mean: 8.7786
    max:  51.5272
    min:  0.38742
  plan: Pred
    mean: 7.1768
    max:  42.0102
    min:  0.52644
  plan: 0T
    mean: 6.7613
    max:  38.3111
    min:  0.28805

"Brainstem" (37.0312 cubic cm.)
  plan: GT
    mean: 31.6635
    max:  70.4023
    min:  1.588
  plan